In [1]:
import plotly.graph_objects as go
from datetime import date, datetime, timedelta
import pandas as pd

In [72]:
itinerarios_bases = pd.read_parquet('../Data/Itinerarios/itinerarios_bases.parquet')
itinerarios_bases["unplug_hourTime"] = pd.to_datetime(itinerarios_bases["unplug_hourTime"])
itinerarios_bases = itinerarios_bases[itinerarios_bases["unplug_hourTime"].dt.tz_localize(None)>pd.to_datetime("20190801")]

In [73]:
itinerarios_bases["ageRange"] = itinerarios_bases["ageRange"].replace([0,1,2,3,4,5,6],["Indeterminado","0-16","17-18", "19-26", "27-40", "41-65", "66+"])
itinerarios_bases

,idplug_base,user_type,idunplug_base,travel_time,idunplug_station,ageRange,idplug_station,unplug_hourTime,zip_code,return_date,...,Barrio_Salida,Número de Plazas_Salida,Latitud_Salida,Longitud_Salida,Número_Llegada,Distrito_Llegada,Barrio_Llegada,Número de Plazas_Llegada,Latitud_Llegada,Longitud_Llegada
131987,2,1,20,328,3,Indeterminado,43,2019-08-01 01:00:00+00:00,nan,2019-08-01 06:28:00.000019200+00:00,...,07-02 ARAPILES,18.0,40.430321,-3.707212,43,01 CENTRO,01-02 EMBAJADORES,24.0,40.410088,-3.702545
131988,10,1,23,263,55,Indeterminado,133,2019-08-01 01:00:00+00:00,nan,2019-08-01 05:23:00.000009600+00:00,...,01-05 UNIVERSIDAD,24.0,40.424072,-3.701930,133,06 TETUÁN,06-02 CUATRO CAMINOS,24.0,40.447166,-3.700029
131989,5,1,7,446,16,Indeterminado,60,2019-08-01 01:00:00+00:00,nan,2019-08-01 08:25:59.999980800+00:00,...,01-05 UNIVERSIDAD,21.0,40.422943,-3.707495,60,03 RETIRO,03-05 JERÓNIMOS,24.0,40.419453,-3.688199
131990,22,1,20,245,29,27-40,33,2019-08-01 01:00:00+00:00,28014,2019-08-01 05:05:00.000009599+00:00,...,01-03 CORTES,24.0,40.416245,-3.695801,33,01 CENTRO,01-06 SOL,21.0,40.416477,-3.703210
131991,22,1,15,297,4,Indeterminado,163,2019-08-01 01:00:00+00:00,nan,2019-08-01 05:57:00+00:00,...,01-05 UNIVERSIDAD,24.0,40.428589,-3.702674,163,02 ARGANZUELA,02-02 ACACIAS,21.0,40.403633,-3.706437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1796825,17,1,24,1433,204,27-40,180,2019-12-31 23:00:00+00:00,28045,2020-01-01 22:53:00.000009600+00:00,...,06-01 BELLAS VISTAS,24.0,40.447662,-3.704170,180,02 ARGANZUELA,02-07 ATOCHA,24.0,40.394127,-3.675924
1796826,1,1,8,2727,4,Indeterminado,99,2019-12-31 23:00:00+00:00,nan,2020-01-02 20:27:00+00:00,...,01-05 UNIVERSIDAD,24.0,40.428589,-3.702674,99,04 SALAMANCA,04-02 GOYA,24.0,40.426105,-3.673964
1796827,6,1,9,898,77,27-40,174,2019-12-31 23:00:00+00:00,28012,2020-01-01 13:57:59.999990400+00:00,...,03-01 PACÍFICO,22.0,40.406334,-3.670774,174,02 ARGANZUELA,02-02 ACACIAS,27.0,40.397415,-3.706377
1796828,24,1,18,15890,216,27-40,216,2019-12-31 23:00:00+00:00,28005,2020-01-11 23:49:59.999980799+00:00,...,10-02 PUERTA DEL ÁNGEL,24.0,40.414051,-3.728895,216,10 LATINA,10-02 PUERTA DEL ÁNGEL,24.0,40.414051,-3.728895


In [74]:
ageRange_1 = itinerarios_bases[itinerarios_bases["user_type"] == 1].groupby("ageRange").size().reset_index(name='Count')
ageRange_2 = itinerarios_bases[itinerarios_bases["user_type"] == 2].groupby("ageRange").size().reset_index(name='Count')

In [75]:
ageRange_1 = itinerarios_bases[itinerarios_bases["user_type"] == 1].groupby("ageRange").agg({"travel_time":"mean"})
ageRange_2 = itinerarios_bases[itinerarios_bases["user_type"] == 2].groupby("ageRange").agg({"travel_time":"mean"})

In [76]:
ageRange_2

,travel_time
ageRange,
0-16,2024.500000
17-18,2689.000000
19-26,2374.475000
27-40,1825.108614
41-65,1769.452991
66+,15436.000000
Indeterminado,2390.996792


In [81]:
# traveltime ticks
traveltime_ticks = list(range(int(round(-max(max(ageRange_1["travel_time"]), max(ageRange_2["travel_time"])),-3)), int(round(max(max(ageRange_1["travel_time"]), max(ageRange_2["travel_time"])),-3)), 20))


# Creating instance of the figure
fig = go.Figure(go.Bar(y= ageRange_1.index, x = round(ageRange_1["travel_time"]/60), 
                     name = 'Usuarios Anuales', 
                     orientation = 'h',
                      marker_color = "#18bc9c"))

  
# Adding Male data to the figure
fig.add_trace((go.Bar(y= ageRange_2.index, x = round(ageRange_2["travel_time"]*-1/60), 
                     name = 'Usuarios Ocasionales', 
                     orientation = 'h',
                     marker_color = "#395B64")))
  
# Updating the layout for our graph
fig.update_layout(title = 'Population Pyramid of Ages by user type',
                  plot_bgcolor='rgba(0,0,0,0)',
                 barmode='relative',
                bargap = 0.0, bargroupgap = 0,
                xaxis = dict(tickvals = traveltime_ticks,    
                title = 'Media duración de trayectos',
                title_font_size = 14),
                  yaxis = dict(ticktext =["Indeterminado","0-16","17-18", "19-26", "27-40", "41-65", "66+"])
                 )
fig.update_xaxes(gridcolor="lightgrey")
  
fig.show()